# Optiver Quant Researcher Test
## Answers to questions 1, 2 and 3.

### Completed by Nikita Izmailov

In [344]:
import numpy as np
import random

## Question 1:
- Question: if the food is located on east-west lines 20cm to the north and 20cm to the south, as well as on north-south lines 20cm to the east and 20cm to the west from the anthill, how long will it take the ant to reach it on average?
- Answer: The simulation run on 20 trials with 100,000 iterations for average time taken and standard deviation. 
- This is a closed boundary: |x| >= 20 and |y| >= 20 with such condition.

In [338]:
class Ant:
    def __init__(self, speed=10, coords=[0,0]):
        # speed 10 cm per second.
        self.speed = speed
        # equal probability of choosing direction = 1/4
        # 1 = north, 2 = east, 3 = south, 4 = west
        self.directions = ["north","east","south","west"]
        # x, y coordinates of the ant at the start
        self.coords = np.array(coords)
    
    def chooseDirection(self):
        direction = random.randint(0, len(self.directions) - 1)
        return self.directions[direction]
    
    def move(self):
        sec = 1
        direction = self.chooseDirection()
        if direction == "north":
            val = self.coords[1] + self.speed
            self.coords[1] = val
        elif direction == "south":
            val = self.coords[1] - self.speed
            self.coords[1] = val
        elif direction == "east":
            val = self.coords[0] + self.speed
            self.coords[0] = val
        elif direction == "west":
            val = self.coords[0] - self.speed
            self.coords[0] = val
        
        return sec
    
    def reachFood(self, foodCoords = set([20, -20])):
        secondsCounter = 0
        self.coords = np.array([0,0])
        while self.coords[0] not in foodCoords and self.coords[1] not in foodCoords:
            makeRunSec = self.move()
            secondsCounter += makeRunSec
        storeArray = [self.coords, secondsCounter]
        
        return storeArray

random.seed(0)
    
def getMeanAndStd(X):
    mean = sum(X)/float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    std = np.sqrt(tot/len(X))
    return mean, std
        
    
# simulate 20 trials with 100,000 runs each and take an average and std.
def findAverageTimeTakenToReach(game, numTrials, numRuns):
    timeTaken = []
    for i in range(numTrials):
        tempTime = []
        for j in range(numRuns):
            timeFood = game.reachFood()[1]
            tempTime.append(timeFood)
        meanVal = np.mean(tempTime)
        timeTaken.append(meanVal)
    
    # Calculate Mean and Standard Deviation for the array of mean times taken
    mean, std = getMeanAndStd(timeTaken)
    
    return mean, std

In [339]:
game = Ant()

timeTakenOnAverage = findAverageTimeTakenToReach(game, 20, 100000)

print("\nSimulate time taken to reach food by Ant on 20 trials with 100,000 runs")
print("Average Time Taken:", str(round(timeTakenOnAverage[0], 2)) + "±" + str(round(timeTakenOnAverage[1], 2)) + " seconds")


Simulate time taken to reach food by Ant on 20 trials with 100,000 runs
Average Time Taken: 4.5±0.01 seconds


### Explaining the Answer to Question 1:
As per the simulation I ran, we can see that an ant on average takes 4.5 seconds with tiny standard deviation for a square boundary, shows small variance/dispersion in data, so you can be very sure that the time ant travels is very consistent with the mean.

## Question 2:
- What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?
- Hence the boundary condition is x + y = 10.

In [347]:
class AntQuestion2(Ant):
    def __init__(self):
        super().__init__()
    
    def reachFoodForUnboundedZone(self, secondsToRun):
        secondsCounter = 0
        self.coords = np.array([0,0])
        reachedFood = False
        while secondsCounter < secondsToRun:
            makeRunSec = self.move()
            secondsCounter += makeRunSec
            if np.nansum(self.coords) == 10:
                reachedFood = True
                break
        storeArray = [self.coords, secondsCounter, reachedFood]
        
        return storeArray
    
antQs2= AntQuestion2()

In [348]:
def findAverageTimeTakenForUnbounded(game, numTrials, numRuns, seconds=[100,1000,10000]):
    arrayOfTimeAndSuccessAnts = []
    for time in seconds:
        timeTaken = []
        antsMadeIt = []
        for i in range(numTrials):
            successfulAnts = 0
            tempTime = []
            for j in range(numRuns):
                coords, timeFood, hasReached = game.reachFoodForUnboundedZone(time)
                if hasReached == True:
                    successfulAnts += 1
                    tempTime.append(timeFood)
            meanVal = np.mean(tempTime)
            timeTaken.append(meanVal)
            antsMadeIt.append(successfulAnts / numRuns)
    
        # Calculate Mean and Standard Deviation for the array of mean times taken
        meanTime, stdTime = getMeanAndStd(timeTaken)
        meanAnts, stdAnts = getMeanAndStd(antsMadeIt)
        arrayOfTimeAndSuccessAnts.append([time, meanTime, stdTime, meanAnts, stdAnts])
    
    return arrayOfTimeAndSuccessAnts

In [349]:
arrayOfData = findAverageTimeTakenForUnbounded(antQs2, 10, 1000)

In [350]:
# Results for Question 2:
for data in arrayOfData:
    print("\nSimulate time taken to reach food by Ant on 10 trials with 1000 runs")
    print(str(data[0]) + " steps/seconds allowed for ant to reach the food")
    print("Average Time Taken:", str(round(data[1], 2)) + "±" + str(round(data[2], 2)) + " seconds")
    print("In this time on average:", str(round(data[3] * 100, 2)) + "±" + str(round(data[4] * 100, 2)) + "% have reached the food")


Simulate time taken to reach food by Ant on 10 trials with 1000 runs
100 steps/seconds allowed for ant to reach the food
Average Time Taken: 7.69±0.42 seconds
In this time on average: 92.27±1.0% have reached the food

Simulate time taken to reach food by Ant on 10 trials with 1000 runs
1000 steps/seconds allowed for ant to reach the food
Average Time Taken: 24.46±3.16 seconds
In this time on average: 97.65±0.52% have reached the food

Simulate time taken to reach food by Ant on 10 trials with 1000 runs
10000 steps/seconds allowed for ant to reach the food
Average Time Taken: 72.7±16.68 seconds
In this time on average: 99.27±0.3% have reached the food


### Explaining the Answer to Question 2:
As per the simulation I ran, we can see that not all ants can reach the diagonal line with formula y=-x+10, this is because this is an open boundary. But, an ant on average takes 72.7 seconds with ±16.68 seconds standard deviation for a open boundary. The standard deviation is quite big this is because it is affected by extreme values where ant just wonders off to wrong direction and might take much more time to reach the diagonal line, or not reach it at all.

## Question 3:
- Can you write a program that comes up with an estimate of average time to find food for any closed boundary around the anthill? What would be the answer if food is located outside an defined by ( (x – 2.5cm) / 30cm )2 + ( (y – 2.5cm) / 40cm )2 < 1 in coordinate system where the anthill is located at (x = 0cm, y = 0cm)? Provide us with a solution rounded to the nearest integer.

In [353]:
class AntQuestion3(Ant):
    def __init__(self):
        super().__init__()
    
    def reachFoodForCircle(self, secondsToRun):
        secondsCounter = 0
        self.coords = np.array([0,0])
        # Reached food if ant reached the perimeter of the circle or outside of the circle
        reachedFood = False
        
        while not reachedFood and secondsCounter < secondsToRun:
            makeRunSec = self.move()
            secondsCounter += makeRunSec
            
            reachedFood = np.square((self.coords[0] - 2.5) / 30) + np.square((self.coords[1] - 2.5) / 40) >= 1
        
        storeArray = [self.coords, secondsCounter, reachedFood]
        
        return storeArray
        
    
antQs3 = AntQuestion3()

In [357]:
# simulate 20 trials with 100,000 runs each and take an average and std.
# Updated the function to use a different method
def findAverageTimeTakenForCircle(game, numTrials, numRuns, seconds=[100,1000,10000]):
    arrayOfTimeAndSuccessAnts = []
    for time in seconds:
        timeTaken = []
        antsMadeIt = []
        for i in range(numTrials):
            successfulAnts = 0
            tempTime = []
            for j in range(numRuns):
                coords, timeFood, hasReached = game.reachFoodForCircle(time)
                if hasReached == True:
                    successfulAnts += 1
                    tempTime.append(timeFood)
            meanVal = np.mean(tempTime)
            timeTaken.append(meanVal)
            antsMadeIt.append(successfulAnts / numRuns)
    
        # Calculate Mean and Standard Deviation for the array of mean times taken
        meanTime, stdTime = getMeanAndStd(timeTaken)
        meanAnts, stdAnts = getMeanAndStd(antsMadeIt)
        arrayOfTimeAndSuccessAnts.append([time, meanTime, stdTime, meanAnts, stdAnts])
    
    return arrayOfTimeAndSuccessAnts

In [358]:
arrayOfDataQS3 = findAverageTimeTakenForCircle(antQs3, 10, 10000)

In [362]:
# Results for Question 3:
for data in arrayOfDataQS3:
    print("\nSimulate time taken to reach food by Ant on 10 trials with 10,000 runs")
    print(str(data[0]) + " steps/seconds allowed for ant to reach the food")
    print("Average Time Taken:", str(round(data[1])) + "±" + str(round(data[2], 2)) + " seconds")
    print("In this time on average:", str(round(data[3] * 100, 2)) + "±" + str(round(data[4] * 100, 2)) + "% have reached the food")


Simulate time taken to reach food by Ant on 10 trials with 10,000 runs
100 steps/seconds allowed for ant to reach the food
Average Time Taken: 14±0.1 seconds
In this time on average: 100.0±0.01% have reached the food

Simulate time taken to reach food by Ant on 10 trials with 10,000 runs
1000 steps/seconds allowed for ant to reach the food
Average Time Taken: 14±0.11 seconds
In this time on average: 100.0±0.0% have reached the food

Simulate time taken to reach food by Ant on 10 trials with 10,000 runs
10000 steps/seconds allowed for ant to reach the food
Average Time Taken: 14±0.09 seconds
In this time on average: 100.0±0.0% have reached the food


### Explaining the Answer to Question 3:
As per the simulation I ran, we can see that an ant on average takes 14 seconds with tiny standard deviation for a circle boundary, shows small variance/dispersion in data, so you can be very sure that the time ant travels is very consistent with the mean.